# Analyzing borrowers’ risk of defaulting

Here, I have prepared a report for a bank’s loan division, which analyzes whether a customer’s marital status and number of children has an impact on loan repayment.


In [1]:
import pandas as pd
import numpy as np

credit_scoring_eng = pd.read_csv('/datasets/credit_scoring_eng.csv')

credit_scoring_eng.groupby('days_employed')['total_income']


##  Description of the data

****
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan



In [2]:
credit_scoring_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


The [days_employed] column has negative values, which may be addressed by creating a separate [days_unemployed] column, 
    if this step wll  optimize further data analysis and clarity. 
    
[days_employed] and [total_income] both have missing quantitative values, which may be resolved by applying 
    representative values such as the median values of the releveant corresponding educational and socieconomic sample. 
    
[days_employed] and [total_income] should be further subdivided into ranges of days and total income to make
 the data more manageable.
    

[dob_years] and  [family_status_id] column names may be ambiguous, 
and, if upon further analysis this ambiguity is, in fact, counterproductive, the columns should be renamed for clarity;

The age should be grouped in ranges to make the data more manageable

There are varying cell formats in  in the [education] column; that is, some are capitalized, 
whilst other cells are lower case. This may be remedied by applying a function to make all values lower case.

The row indexes are unnamed, which may indicate that this dataframe is an amalgam of a raw, unorganized data set.  




[days_employed] and [total_income] both have missing quantitative values, which may be correlated variables, 
and, are likely significant variables of interest.

In [4]:
credit_scoring_eng.iloc[:,1] 

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [5]:
credit_scoring_eng.isnull().sum()/len(credit_scoring_eng)

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

In [6]:
mis_values = credit_scoring_eng.isnull().sum().to_frame('missing_values')
mis_values['%'] = round(credit_scoring_eng.isnull().sum()/len(credit_scoring_eng),3)# What does the ",3" parameter do?

mis_values.sort_values(by='%', ascending=False)

,missing_values,%
days_employed,2174,0.101
total_income,2174,0.101
children,0,0.000
dob_years,0,0.000
education,0,0.000
education_id,0,0.000
family_status,0,0.000
family_status_id,0,0.000
gender,0,0.000
income_type,0,0.000


In [7]:
credit_scoring_eng.isnull().sum()/len(credit_scoring_eng)

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Filtering data in the "days_employed" column 

In [8]:
credit_scoring_eng['days_employed'].value_counts()

-327.685916     1
-1580.622577    1
-4122.460569    1
-2828.237691    1
-2636.090517    1
               ..
-7120.517564    1
-2146.884040    1
-881.454684     1
-794.666350     1
-3382.113891    1
Name: days_employed, Length: 19351, dtype: int64

The following test indicates that the number of missing values is symmetrical; 
that is, the number of missing values in the ['total_income'] column equals to the umber of missing values in the ['days_employed'] column. 

In [9]:
credit_scoring_eng['days_employed'].isna().sum() == credit_scoring_eng['total_income'].isna().sum()

True

In [10]:
credit_scoring_eng['days_employed'].isna().sum() # >>>>> 2174 clients = 21525 entries total - 19351 ['days_employed']        
credit_scoring_eng['total_income'].isna().sum()# >>>>>>>>2174 clients = 21525 entries total - 19351 ['total_income']

2174

There are implicit duplicate categorical values in the ['purpose'] column.


In [11]:
credit_scoring_eng['purpose'].unique()

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

Checking for the number of rows with a negative number of "days employed."

In [12]:
len(credit_scoring_eng.index[credit_scoring_eng['days_employed'] < 0])

15906

15906 clients negative values days employed checking for pattern in rows with negative [days_employed] values. 

In [13]:
credit_scoring_eng.groupby('income_type').size()

income_type
business                        5085
civil servant                   1459
employee                       11119
entrepreneur                       2
paternity / maternity leave        1
retiree                         3856
student                            1
unemployed                         2
dtype: int64

In [14]:
credit_scoring_eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [15]:
len(credit_scoring_eng[credit_scoring_eng.duplicated(subset='days_employed')])

2173

Checking to see whether the negative days employed has anything to do with being unemployed.

One can conclude that there is no connection between negative days and being unemployed because the only two client that reported being unemployed, have positive values in the days employed column; although the values are anomolous, because the number of days is the equivalent of more than 800 years.

In [16]:
credit_scoring_eng[credit_scoring_eng['income_type'] == 'unemployed']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,secondary education,1,married,0,M,unemployed,1,9593.119,buying property for renting out
14798,0,395302.838654,45,Bachelor's Degree,0,civil partnership,1,F,unemployed,0,32435.602,housing renovation


In [17]:
# Checking distribution
#credit_scoring_eng.isna().sum().sum()#the total number of missing values in the dataframe is 4240.
#credit_scoring_eng.decsribe()
credit_scoring_eng['income_type'].isna().sum()

0

The proportion of missing values in the dataframe to the whole data set is about 20%.

The values may be missing because some clients did not indicate explicitly that they are unemployed, and instead indicate the days since their last employment ended. Also, some retirees may have ommitted there employment income because they are technially unemployed.


Checking the statistics in the whole dataset

In [18]:
credit_scoring_eng.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


The percentage of the missing values compared to the whole dataset is about 20%.

In [19]:
credit_scoring_eng.isna().sum().sum() / len(credit_scoring_eng)

0.20199767711962835

In [20]:
# Check for other reasons and patterns that could lead to missing values

credit_scoring_eng['days_employed'].describe()


count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

We can conclude that the missing values are the result of a mistake, and are not random errors.

## Data transformation





Checking all values in education column to check if and what spellings will need to be fixed.

In [21]:

credit_scoring_eng['education'].unique()


array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

Making all row values in "education", "purpose" columns  lowercase.

In [22]:
credit_scoring_eng['education'] = credit_scoring_eng['education'].str.lower()
credit_scoring_eng['purpose'] = credit_scoring_eng['purpose'].str.lower()
credit_scoring_eng['education'].duplicated().describe()


count     21525
unique        2
top        True
freq      21520
Name: education, dtype: object

Confirming there are no duplicates.

In [23]:
len(credit_scoring_eng.index[credit_scoring_eng['education'] == credit_scoring_eng['education'].duplicated()])

0

In [24]:
credit_scoring_eng['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

Checking children column values.

In [25]:
credit_scoring_eng['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [26]:
credit_scoring_eng['children'].describe()


count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

Some clients report 20 kids.

In [27]:
credit_scoring_eng['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Counting the number of clients that report having 20 kids.

In [28]:
len(credit_scoring_eng.index[credit_scoring_eng['children'] == 20])

76

About 0.35% percent of all clients report 20 kids, which tends to show that there is an error, that must be remedied with the average or median number of kids.

In [29]:
len(credit_scoring_eng.index[credit_scoring_eng['children'] == 20]) / len(credit_scoring_eng)

0.0035307781649245064

In [30]:
credit_scoring_eng.groupby('family_status')['children'].value_counts()

family_status      children
civil partnership   0          2752
                    1           998
                    2           344
                    3            56
                    20           12
                    4             8
                   -1             5
                    5             2
divorced            0           784
                    1           312
                    2            81
                    3            11
                   -1             4
                    20            2
                    4             1
married             0          7500
                    1          2982
                    2          1535
                    3           249
                    20           49
                   -1            29
                    4            29
                    5             7
unmarried           0          2265
                    1           449
                    2            75
                    20            9


[Are there any strange things in the column? If yes, how high is the percentage of problematic data? How could they have occurred? Make a decision on what you will do with this data and explain you reasoning.]

47 clients reported negative number for kids

In [31]:
len(credit_scoring_eng.index[credit_scoring_eng['children'] < 0])

47

Calculating percentage of problematic values in kids column 

    About 0.6% of values in the children column are problematic.

In [32]:
problem_child_neg = len(credit_scoring_eng.index[credit_scoring_eng['children'] < 0]) # number of [children] columns with negative values  
problem_child_20 = len(credit_scoring_eng.index[credit_scoring_eng['children'] == 20])# 76 cleints report having 20 kids
total_child = credit_scoring_eng['children'].count()
(problem_child_neg + problem_child_20) / total_child


0.005714285714285714

Getting the absolute value of the negative values, and dropping columns with 20 kids because this only comprises 76 clients, which is negligible for a dataset of this size.

In [33]:
credit_scoring_eng['children'] = credit_scoring_eng['children'].abs()

In [34]:
credit_scoring_eng = credit_scoring_eng.loc[credit_scoring_eng['children'] != 20]

Confirming that the negative '20' values have been fixed. 

In [35]:
credit_scoring_eng['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [36]:
credit_scoring_eng = credit_scoring_eng.loc[credit_scoring_eng['children'] != 20]# dropped clients with 20 kids because it comprises a negligible proportion of dataframe
#***Tests for ['children'] column***
#credit_scoring_eng.iloc[291]
#credit_scoring_eng.iloc[705]
#credit_scoring_eng.iloc[11604]
#credit_scoring_eng.iloc[19373]


In [37]:
len(credit_scoring_eng.index[credit_scoring_eng['children'] == 20])# clients with 20 kids dropped

0

In [38]:
len(credit_scoring_eng.index[credit_scoring_eng['children'] < 0]) # Confirmed that all negative value converted to absolute values

0

Checking the data in the `days_employed` column. 

In [39]:
credit_scoring_eng['days_employed'].describe()

count     19284.000000
mean      63141.233527
std      140910.281638
min      -18388.949901
25%       -2747.876441
50%       -1204.164714
75%        -290.206221
max      401755.400475
Name: days_employed, dtype: float64

The maximum value for the ['days_employed'] column is 401755, the equivalent of 1100 years. There are negaive values and floating-point numbers in the column, which is likely innaproriate for counting days.

In [40]:
credit_scoring_eng.loc[credit_scoring_eng['days_employed'].idxmax()]

children                              0
days_employed             401755.400475
dob_years                            56
education           secondary education
education_id                          1
family_status           widow / widower
family_status_id                      2
gender                                F
income_type                     retiree
debt                                  0
total_income                  28204.551
purpose              housing renovation
Name: 6954, dtype: object

15846 clients indicated they worked 45 years or less.

In [41]:
len(credit_scoring_eng.index[credit_scoring_eng['days_employed'] < 16500])

15846

 15846 clients reported negative values for days worked.

In [42]:
len(credit_scoring_eng[credit_scoring_eng['days_employed'] < 0])

15846

3438 clients that worked more than 45 years

In [43]:
len(credit_scoring_eng.index[credit_scoring_eng['days_employed'] > 16500])

3438

3438 clients that worked more than 50 years

In [44]:
len(credit_scoring_eng.index[credit_scoring_eng['days_employed'] > 18250])

3438

Checking for missing values in days employed column

In [45]:
credit_scoring_eng['days_employed'].isna().sum() #2165 clients with missing values

2165

Fixing problematic data in days_employed column.

In [46]:
import warnings
warnings.filterwarnings("ignore")

workdays_median = int(credit_scoring_eng['days_employed'].median())

credit_scoring_eng['days_employed'] = credit_scoring_eng['days_employed'].fillna(workdays_median)
#Replaced mising values with median.

credit_scoring_eng['days_employed'] = credit_scoring_eng['days_employed'].abs()
#Converted values to absolute values.

credit_scoring_eng['days_employed'].fillna(value=days_empl_median, inplace=True )
# replace missing values wityh median values in [days_employed] column


NameError: name 'days_empl_median' is not defined

In [ ]:
credit_scoring_eng['days_employed'] = np.where((credit_scoring_eng.days_employed > 20000), workdays_median, credit_scoring_eng.days_employed)
#replaced extreme number of days e.g. the equivalent of ~1000 years

*Confirming whether problematic data is fixed.*

In [ ]:
credit_scoring_eng['days_employed'].isna().sum()

In [ ]:
credit_scoring_eng['days_employed'].describe()

*Checking for issues in the client's age in 'dob_years' column.* 

In [ ]:
credit_scoring_eng['dob_years'].unique()

In [ ]:
len(credit_scoring_eng.index[credit_scoring_eng['dob_years'] == 0])# 100 clients didn't report their age 
                                                                    

In [ ]:
dob_prob_pct =  len(credit_scoring_eng.index[credit_scoring_eng['dob_years'] == 0]) / len(credit_scoring_eng.index[credit_scoring_eng['dob_years']])
dob_prob_pct
#about 0.5% of clients reported 0 as age.

Using the average age for missing values will be more approriate because it will be more representative of the dataset than the median.

In [ ]:
age_avg = int(credit_scoring_eng['dob_years'].mean())

credit_scoring_eng['dob_years'].replace({ 0: int(age_avg)}, inplace=True)#replaced 0 with mean age of 42

*Confirming the "0" value has been remedied.*

In [ ]:

len(credit_scoring_eng.loc[credit_scoring_eng['dob_years'] == 0])

Checking the `family_status` column. 

In [ ]:
credit_scoring_eng['family_status'].unique()

In [ ]:
credit_scoring_eng['family_status'].isna().sum()

In [ ]:
credit_scoring_eng['family_status'].describe()

In [ ]:
# Address the problematic values in `family_status`, if they exist

credit_scoring_eng['family_status'].value_counts()
#credit_scoring_eng['family_status'].unique()

Checking the `gender` column. 

In [ ]:
credit_scoring_eng['gender'].unique()

In [ ]:
credit_scoring_eng[credit_scoring_eng['gender'] == 'XNA'] 
#politically loaded question in the west of whether there's something other than male and femal gender

In [ ]:
credit_scoring_eng.loc[credit_scoring_eng['gender'] == 'M'].count() #7279 male clients
credit_scoring_eng.loc[credit_scoring_eng['gender'] == 'F'].count() #14174 female clients
credit_scoring_eng[credit_scoring_eng['gender'] == 'XNA'] 
credit_scoring_eng.info() # 7279 + 14174 = 21,453 No row is missing a gender value, except one.

*Fixing the gender column.*

In [ ]:
credit_scoring_eng.index[credit_scoring_eng['gender'] == 'XNA'] # row with XNA gender is index 10701
credit_scoring_eng = credit_scoring_eng.drop(10701)# deletes row with gender 'XNA'

*Confirming gender column issues are fixed.*

In [ ]:
len(credit_scoring_eng[credit_scoring_eng['gender'] == 'XNA']) # deleted row

*Checking `income_type` column.*

In [ ]:
credit_scoring_eng['income_type'].describe()

In [ ]:
credit_scoring_eng['income_type'].unique()

In [ ]:
credit_scoring_eng['income_type'].isna().sum()

In [ ]:
credit_scoring_eng['income_type'].value_counts()

*Checking for duplicates in dataframe*

In [ ]:
credit_scoring_eng.duplicated().sum()

In [ ]:
duplicated = credit_scoring_eng[credit_scoring_eng.duplicated()]
duplicated

In [ ]:
#credit_scoring_eng.iloc[2849] == credit_scoring_eng.iloc[3290]

#credit_scoring_eng.iloc[20297] == credit_scoring_eng.iloc[20662]

#credit_scoring_eng.duplicated().sum()

Removing Dupliates

In [ ]:
credit_scoring_eng = credit_scoring_eng.drop_duplicates().reset_index(drop=True)

In [ ]:
credit_scoring_eng.duplicated().sum()

In [ ]:
credit_scoring_eng.info()

*Here, 71 duplicate indexes have been removed, which comprises 0.3% of the total dataframe.* 


# Working with missing values

[To speed up working with some data, you may want to work with dictionaries for some values, where IDs are provided. Explain why and which dictionaries you will work with.]

### Restoring missing values in `total_income`

Renaming dob_years to age:

In [ ]:
credit_scoring_eng = credit_scoring_eng.rename(columns={'dob_years': "age"})

In [ ]:
credit_scoring_eng.columns#confirming column renamed to 'age'

In [ ]:
credit_scoring_eng['total_income'].describe()

In [ ]:
credit_scoring_eng['age'] = credit_scoring_eng['age'].astype(int)# coverted age to integers

Applying function to organize age groups in generations and creating a new 'generation' column

In [ ]:
def age_groups(years):
    
    if 8 < years < 25:
        return '18-24'
    if 24 < years < 41:
        return '25-40'
    if 40 < years < 57:
        return '41-56'
    if 56 < years < 67:
        return '57-66'
    if  66 < years < 76:
        return '67-75'
    if  75 < years < 94:
        return '76-93'
    return 'age unknown'


credit_scoring_eng['generation'] = credit_scoring_eng.age.apply(age_groups)#grouped ages by generation to make dataframe more organized

In [ ]:
credit_scoring_eng.columns# confirming generation column has been created

Filling missing values in total_income column with median income.

In [ ]:
income_median = credit_scoring_eng['total_income'].median()

credit_scoring_eng['total_income'] = credit_scoring_eng['total_income'].fillna(int(income_median))

In [ ]:
credit_scoring_eng['total_income'].isna().sum()

In [ ]:
credit_scoring_eng.head()

Conforming the generation column function is working properly:

In [ ]:
#credit_scoring_eng.columns # changed dob_years to age
gens = credit_scoring_eng.groupby(['generation', 'gender']).agg({'debt': 'sum'})
gens

In [ ]:
# Creating new column based on function
credit_scoring_eng['total_income'].median()


Factors that influence income:

In [ ]:
grouped_df_education = credit_scoring_eng.groupby('education').agg({'total_income': ['median', 'mean',  'min', 'max']})
grouped_df_education

In [ ]:
grouped_df_family_stat = credit_scoring_eng.groupby('family_status').agg({'total_income': ['median', 'mean',  'min', 'max']})
grouped_df_family_stat

In [ ]:
grouped_df_gender = credit_scoring_eng.groupby('gender').agg({'total_income': ['median', 'mean',  'min', 'max']})
grouped_df_gender

In [ ]:
grouped_df_income_type = credit_scoring_eng.groupby('income_type').agg({'total_income': ['median', 'mean',  'min', 'max']})
grouped_df_income_type 

***Mean Values***

In [ ]:
#Grouped by age: $24900
#Grouped by education: $26870.50
#Grouped by age: $25878.40
#Grouped by gender: F	$24506.897740
                   #M	$30193.488521
    

In [ ]:
# Grouped by income type

In [ ]:
#grouped_df_age.describe()
#grouped_df_education.describe()
#grouped_df_family_stat.describe()
#grouped_df_gender

Gender and income-type markedly influence income.       

In [ ]:
# Checking the number of entries in the columns

In [ ]:
credit_scoring_eng['total_income'].describe()

In [ ]:
credit_scoring_eng['total_income'].isna().sum() == credit_scoring_eng['days_employed'].isna().sum()

###  Restoring values in `days_employed`

Distribution of `days_employed` medians based on identified parameters

In [ ]:
data_grouped_d_empl_median = credit_scoring_eng.groupby(['income_type', 'gender', 'debt']).agg({'days_employed': 'median'})
data_grouped_d_empl_mean =  credit_scoring_eng.groupby(['income_type', 'gender', 'debt']).agg({'days_employed': 'mean'}) 
data_grouped_d_empl_median

In [ ]:
data_grouped_d_empl_generation_mean =  credit_scoring_eng.groupby(['generation', 'gender']).agg({'days_employed': 'mean', 'debt': 'sum'})
data_grouped_d_empl_generation_median = credit_scoring_eng.groupby(['generation', 'gender']).agg({'days_employed': 'median', 'debt': 'sum'})
data_grouped_d_empl_generation_median

In [ ]:
data_grouped_d_empl_fam_stat_mean =  credit_scoring_eng.groupby(['family_status', 'gender', 'debt']).agg({'days_employed': 'mean'})
data_grouped_d_empl_fam_stat_median = credit_scoring_eng.groupby(['family_status', 'gender', 'debt']).agg({'days_employed': 'median'})
data_grouped_d_empl_fam_stat_median

Distribution of `days_employed` means based on identified parameters

# Family Status / Gender / Debt:

In [ ]:
#			days_employed
#family_status	gender	debt	
#civil partnership	F	0	2221.549833
#1	1849.067175
#M	0	1909.715280
#1	1452.369555
#divorced	F	0	2294.504442
#1	1964.130937
#M	0	2104.587216
#1	1663.758442
#married	F	0	2363.065689
#1	1906.176708
#M	0	2143.139645
#1	1789.180760
#unmarried	F	0	1965.226182
#1	1753.034656
#M	0	1550.102812
#1	1292.566716
#widow / widower	F	0	2287.664101
#1	2078.679486
#M	0	1845.726838
#1	1589.485209

# Age / Gender / Debt

In [ ]:
#days_employed
#generation	gender	debt	
#25-40	F	0	3257.812273
#1	3747.840984
#M	0	3498.460011
#1	3943.789394
#31-56	F	0	54930.892025
#1	42321.479467
#M	0	26274.503337
#1	24758.179326
#57-66	F	0	239597.301356
#1	253364.498976
#M	0	166009.142271
#1	113800.421905
#67-75	F	0	301579.403077
#1	305771.498679
#M	0	294264.924119
#1	258008.324704
#9-24	F	0	1377.339466
#1	508.326735
#M	0	605.898204
#1	707.671945

# Income Type / Gender / Debt

In [ ]:
#	days_employed
#income_type	gender	debt	
#business	F	0	1857.433161
#1	1591.263221
#M	0	1737.892368
#1	1348.705277
#civil servant	F	0	3028.018352
#1	3169.096762
#M	0	2762.776949
#1	1708.575287
#employee	F	0	2205.528858
#1	1599.899829
#M	0	1759.121787
#1	1373.095800
#entrepreneur	F	0	520.848083
#M	0	-1204.164714
#paternity / maternity leave	F	1	3296.759962
#retiree	F	0	328548.985374
#1	306370.257129
#M	0	333953.246867
#1	305710.439988
#student	M	0	578.751554
#unemployed	F	0	395302.838654
#M	1	337524.466835

Median values will reflect a more accurate representation of the sample, as a whole, whereby value extremes will not skew a reprentative mean value. However, the mean may be more approrpiate when analyzing values the range age groups. 

Days Employed Median Function

In [ ]:
workdays_median = int(credit_scoring_eng['days_employed'].median())

credit_scoring_eng['days_employed'] = credit_scoring_eng['days_employed'].fillna(workdays_median)
credit_scoring_eng['days_employed'] = credit_scoring_eng['days_employed'].abs()

credit_scoring_eng['days_employed'] = np.where((credit_scoring_eng.days_employed > 20000), workdays_median, credit_scoring_eng.days_employed)
#replaced extreme number of days e.g. the equivalent of ~1000 years

In [ ]:
# Check that the function works
credit_scoring_eng['days_employed'].iloc[6931]# '401755.40047533' changed to median value of '1823'
credit_scoring_eng.describe()
#credit_scoring_eng.info()
#credit_scoring_eng['days_employed'].value_counts()


 `total_income` number of values in column matches the number of values in other ones

In [ ]:
credit_scoring_eng.info()

In [ ]:
credit_scoring_eng.describe()

<div class="alert alert-success" role="alert">
<b>Reviewer's comment v1:</b>
    
Very detailed analisis of missing data. And you correctly identified columns with missing or strange values.
Also very cool that you created functions to partially automate your code.

## Categorization of data

In [ ]:
# Print the values for your selected data for categorization
data_pivot_1 = credit_scoring_eng.pivot_table(index='gender', values='total_income', aggfunc='median')
data_pivot_1

In [ ]:
data_pivot_2 = credit_scoring_eng.pivot_table(index='education', values='total_income', aggfunc='mean')
data_pivot_2

In [ ]:
data_pivot_3 = credit_scoring_eng.pivot_table(index='income_type', values='total_income', aggfunc='median') 
data_pivot_3

In [ ]:
data_pivot_4 = credit_scoring_eng.pivot_table(index='debt', values='total_income', aggfunc='mean')# changed aggfunc to mean because no difference in median
data_pivot_4

In [ ]:
data_pivot_5 = credit_scoring_eng.pivot_table(index='family_status', values='total_income', aggfunc='median')#no difference in total income based on family status, except widowers had the lowest income
data_pivot_5                                                                                                  #and more variation amongst the median values but still negligible differences'

#Total income of a borrwer is a fundamental criterion for assessing a borrower's ability to pay back a loan; and, the two columnns that were typififed by the widest range was when grouping by the means of "income type" and 'gender'.

In [ ]:
data_grouped = credit_scoring_eng.groupby(['income_type', 'gender', 'debt']).agg({'total_income': 'mean'})
data_grouped

In [ ]:
            
data_grouped_income_median = credit_scoring_eng.groupby(['income_type', 'gender', 'debt']).agg({'total_income': 'median'})
data_grouped_income_mean =  credit_scoring_eng.groupby(['income_type', 'gender', 'debt']).agg({'total_income': 'mean'}) 

data_grouped_income_median

In [ ]:
data_grouped_income_generation_mean =  credit_scoring_eng.groupby(['generation', 'gender', 'debt']).agg({'total_income': 'mean'})
data_grouped_income_generation_median = credit_scoring_eng.groupby(['generation', 'gender', 'debt']).agg({'total_income': 'median'})

data_grouped_income_generation_median

In [ ]:
data_grouped_income_fam_stat_mean =  credit_scoring_eng.groupby(['family_status', 'gender', 'debt']).agg({'total_income': 'mean'})
data_grouped_income_fam_stat_median = credit_scoring_eng.groupby(['family_status', 'gender', 'debt']).agg({'total_income': 'median'})

data_grouped_income_fam_stat_median

In [ ]:
            
multi_group_mean =  credit_scoring_eng.groupby(['family_status', 'generation', 'gender', 'debt']).agg({'total_income': 'mean'})
multi_group_median = credit_scoring_eng.groupby(['family_status', 'gender']).agg({'total_income': 'median', 'debt': 'sum'})

multi_group_median 

In [ ]:
purpose_group_median = credit_scoring_eng.groupby(['generation','purpose']).agg({'debt': 'sum', 'purpose': 'count'}) 
purpose_group_median

***Removing Implicit Duplicates in Loan Puprose Column*** 

In [ ]:
def replace_wrong_values_prop(wrong_values, correct_value): # passing a list of wrong values and a string with the correct value on the function's input
    for wrong_value in wrong_values: # looping over misspelled names
        credit_scoring_eng['purpose'] = credit_scoring_eng['purpose'].replace(wrong_value, correct_value) # calling replace() for each wrong name

duplicates = ['building a property', 'building a real estate','buy commercial real estate', 'buy real estate', 'buy residential real estate', 'buying property for renting out', 'construction of own property', 'housing', 'housing renovation', 'housing transactions', 'property', 'purchase of my own house', 'purchase of the house', 'purchase of the house for my family', 'real estate transactions', 'transactions with commercial real estate', 'transactions with my real estate'] # a list of misspelled names
property_purp = 'real property' # correct name
replace_wrong_values_prop(duplicates, property_purp) # function call, replace() will be called twice
 # the new DataFrame without duplicates 
credit_scoring_eng['purpose'].unique()

In [ ]:
def replace_wrong_values_car(wrong_values, correct_value): # passing a list of wrong values and a string with the correct value on the function's input
    for wrong_value in wrong_values: # looping over misspelled names
        credit_scoring_eng['purpose'] = credit_scoring_eng['purpose'].replace(wrong_value, correct_value) # calling replace() for each wrong name

duplicates_car = ['car purchase', 'buying a second-hand car', 'buying my own car', 'cars', 'second-hand car purchase', 'car', 'to own a car', 'purchase of a car', 'to buy a car'] # a list of misspelled names
car_purp = 'car loan' # correct name
replace_wrong_values_car(duplicates_car, car_purp) # function call, replace() will be called twice
 # the new DataFrame without duplicates 
credit_scoring_eng['purpose'].unique()

In [ ]:
def replace_wrong_values_edu(wrong_values, correct_value): # passing a list of wrong values and a string with the correct value on the function's input
    for wrong_value in wrong_values: # looping over misspelled names
        credit_scoring_eng['purpose'] = credit_scoring_eng['purpose'].replace(wrong_value, correct_value) # calling replace() for each wrong name

duplicates_edu = ['supplementary education', 'education', 'to become educated', 'getting an education', 'to get a supplementary education', 'getting higher education', 'profile education', 'university education', 'going to university'] # a list of misspelled names
edu_purp = 'education loan' # correct name
replace_wrong_values_car(duplicates_edu, edu_purp) # function call, replace() will be called twice
 # the new DataFrame without duplicates 
credit_scoring_eng['purpose'].unique()

In [ ]:
def replace_wrong_values_wed(wrong_values, correct_value): # passing a list of wrong values and a string with the correct value on the function's input
    for wrong_value in wrong_values: # looping over misspelled names
        credit_scoring_eng['purpose'] = credit_scoring_eng['purpose'].replace(wrong_value, correct_value) # calling replace() for each wrong name

duplicates_wed = ['to have a wedding', 'having a wedding', 'wedding ceremony'] 
wed_purp = 'wedding loan' # correct name
replace_wrong_values_wed(duplicates_wed, wed_purp) # function call, replace() will be called twice
 # the new DataFrame without duplicates 
credit_scoring_eng['purpose'].unique()

In [ ]:
purpose_group_median_2 = credit_scoring_eng.groupby(['purpose', 'gender']).agg({'debt': 'sum', 'purpose': 'count'}) 
purpose_group_median_2

# Observations
The income type tended to be  inversely correlated, in most samples, with the  outstanding debt.

5236 Bachelors Degrees earn more than 6 Graduate Degrees in this sample and 15112 Seconday Education Clients.

There are significant dipsarities amongst clients' income based on income type

Although women worked more days than men, irrepsective of the age group; nevertheless, they earned less money, and tended to have higher loan deliquency rates. 




## Checking the Hypotheses


**Is there a correlation between having children and paying back on time?**

Checking the children data and paying back on time:

In [ ]:
child_debt_count =  credit_scoring_eng.groupby(['children']).agg({'debt': 'sum', 'children': 'count'})

child_debt_count
# Calculating default-rate based on the number of children

*The foregoing table tends to show that there is positive correlation betweeen having kids and paying back on time; that is, likelihood of default tends to decrease with each successive number of kids.*

**Conclusion**

*Real property loan clients have a 7.2% default rate, and the most applications. Car loan clients have a 9.3% default rate second highets number of applications. Education loan clients have a 9.2% default rate and third in number of applications. Wedding loans comprise the least amount of applications and are typified by a 7.8% default rate.*

*The 25-40 year old age bracket has the most loans. Applications for real estate-related loans have the most applications irrespective of age group.*


<div class="alert alert-warning" role="alert">
<b>Reviewer's comment v1</b>

What conclusions can you make based on the aggregated tables?

**Is there a correlation between family status and paying back on time?**

*Below, the dataframe sustantially confirms one's intuitive predictions when considering a person's ability to make loan payments on time based on their family status, that is, married households tend to have more debt, but their median income is also higher, hich likely explains their relatively low default rates. Interestingly, the dataset suggests that the more children one has, the less likely one will default, which is somewhat counterintuitive, but may be explained on more experience with frugality and capital efficiency. Also, though there are almost twice as many women as men represented in the dataset, 14127 to 7250, repectively; unmarried men are more liley to default than unamrried women, 14% to 6.8% default rate,  which may be explained by males having less responisibilities and urgency to become financially stable to establish a family, or women having additional supplementary financial support.* 

In [ ]:
# Check the family status data and paying back on time

famstat_debt_count =  credit_scoring_eng.groupby(['family_status', 'gender']).agg({'debt': 'sum', 'family_status': 'count'})

famstat_debt_count

***Family Status Default rate*** 

In [ ]:
famstat_debt_count['debt'] / famstat_debt_count['family_status']

*Calculating default-rate based on family status*

In [ ]:
famstat_debt_count_2 =  credit_scoring_eng.groupby(['family_status']).agg({'debt': 'sum', 'family_status': 'count'})
famstat_debt_count_2

**Family Status Default Rate Totals** 

In [ ]:
famstat_debt_count_2['debt'] / famstat_debt_count_2['family_status']

**Conclusion**

Married households and widows/widowers have the lowest default rates because, the former likely enjoys more than one income source, and supplemnetary sources of income to address the relatively capital intensive ambitions of a family, and the latter probably requires a much more ascetic and frugal lifestyle because their partner has died, and the concomitant expenses of family life during child rearing are not required.   

**Is there a correlation between income level and paying back on time?**

Amongst employees, retirees, civil servants, and business owners, there is enough data to infer that the more income one has, the less liley default will occur. Here, retirees enjoy the lowest default rates, followed by civil servants,  business owners, and employees. Employees make the least amount of money, which makes default more likely. However, due to the small sample sizes of "maternity leave," "entrepeneurs," "students" and the "unemployed," there is not enough data to speculate about any correlative tendencies.  

In [ ]:
            
data_grouped_income_median_2 = credit_scoring_eng.groupby(['income_type']).agg({'total_income': 'median', 'debt': 'sum', 'income_type': 'count'})
data_grouped_income_median_2

In [ ]:
data_grouped_income_median_2['debt'] / data_grouped_income_median_2['income_type']

In [ ]:
# Check the income level data and paying back on time
famstat_debt_count = credit_scoring_eng.groupby(['income_type']).agg({'total_income': 'median', 'debt': 'sum', 'income_type': 'count' })
famstat_debt_count
# Calculating default-rate based on income level

In [ ]:
#employee                       11041,,,.....9.5% default rate
#business                        5055,,,.....7.3% default rate
#retiree                         3820,,,.....9.5% default rate
#civil servant                   1455,,,.....5.6% default rate
#entrepreneur                       2,,,.....0% default rate
#unemployed                         2,,,.....50% default rate
#paternity / maternity leave        1,,,.....100% default rate
#student                            1,,,.....0% default rate






#credit_scoring_eng[credit_scoring_eng['income_type'] == 'student']
#credit_scoring_eng['income_type'].value_cou7.3nts
#credit_scoring_eng['income_type'].value_counts()

**How does credit purpose affect the default rate?**

*Default rates by loan purpose tend to show that car loans and education loans have the highest default rates, 9.3% and 9.2%, respectively. This may be due to the more financially precarious nature of debtors' financial circumstances and the corresponding urgency of repayment when compared to expenses for life necessities, such as food and shelter.*

In [ ]:
purpose_debt_count = credit_scoring_eng.groupby(['purpose']).agg({'debt': 'sum', 'purpose': 'count'})
purpose_debt_count 

In [ ]:
purpose_debt_count['debt'] / purpose_debt_count['purpose'] # default rate calculation by purpose

In [ ]:
#default percenatge by purpose
#purpose
#building a property                         0.087237
#building a real estate                      0.077047
#buy commercial real estate                  0.069909
#buy real estate                             0.069579
#buy residential real estate                 0.067881
#buying a second-hand car                    0.074153
#buying my own car                           0.091089
#buying property for renting out             0.080000
#car                                         0.085193
#car purchase                                0.091106
#cars                                        0.092243
#construction of own property                0.065183
#education                                   0.071910
#getting an education                        0.081633
#getting higher education                    0.108235
#going to university                         0.086694
#having a wedding                            0.079843
#housing                                     0.071651
#housing renovation                          0.057851
#housing transactions                        0.074074
#profile education                           0.101382
#property                                    0.066456
#purchase of a car                           0.109890
#purchase of my own house                    0.054839
#purchase of the house                       0.074650
#purchase of the house for my family         0.070644
#real estate transactions                    0.081602
#second-hand car purchase                    0.105809
#supplementary education                     0.083151
#to become educated                          0.095588
#to buy a car                                0.093617
#to get a supplementary education            0.114607
#to have a wedding                           0.076216
#to own a car                                0.098947
#transactions with commercial real estate    0.080495
#transactions with my real estate            0.079872
#university education                        0.089485
#wedding ceremony                            0.081013

#purpose_debt_count

<div class="alert alert-success" role="alert">
<b>Reviewer's comment v1</b>

Great that you used groupby to test hypotheses. You can also apply pivot tables here. 

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

***Total Dataset Composition by Gender***

In [ ]:
gender_count = credit_scoring_eng.groupby(['gender']).agg({'gender': 'count'})
gender_count

# General Conclusion 


***Is there a connection between having kids and repaying a loan on time?***

>***Technical Application***: Converted negative values to absolute, and removed entries with 20 kids.

>***Conclusion***: The pivot table with number of kids and the count of clients with for each categogy may show that there is positive correlation betweeen having kids and paying back on time; that is, likelihood of default tends to decrease with each successive additional number of kids.



***Is there a connection between marital status and repaying a loan on time?***

>***Technical Application***: N/A

>***Conclusion***: Here, in relative magnitudes, married couples tend to have a lower default rate than civil partnerships and the unmarried, coupled with a higher average income; however, married couples still have a higher default rate than divorced individuals and widowed. Commensenisical narratives may offer explanations for such dynammics, e.g., officially married indiividuals may be more socioeconomically stable insofar as they are further in their professional careers with concomitant salaries, and correpsonding subsatntial family expenses, which may lead to financial strain; whereas, the lives of the divorced and unmarried people are more self-sufficient and less ambitious and capital intensive lifestyles.

***Is there a connection between income level and repaying a loan on time?***

>***Technical Application***: Substituted median income of the dataset for missing values.

>***Conclusion***:There may be some correlation between income level and the incidence of loan defaults, but the highest default ratio
was by employees, and civil servants, whose median income was 23202, amongst 11041 clients, adn 14555, respectively. The next highest sample and default rate was the 5055 business clients whose income was $26002. Furthermore, the numbers tend to inidicate there is an inverse correlation between income level and default rate.

***How do different loan purposes affect on-time loan repayment?***

>***Technical Application***: Applied a function to remove implicit duplicates in the 'loan purpose' column.

>***Conclusion***: Although the analysis is more nuanced than this data set permits because macroeconomic factors, monetary policy, inflation, unemployment rates all have a substantial impact on overall solvency and ability to meet financial obligations; nonetheless, these data tend to indicate that at this bank, most people apply for real estate-related loans, with nore than 4000 applications in the , but have the lowest deliquency rate amongst loan purposes. Clients seek almost an equal amount of car and education loans, and are attended by the highest default rates.  
Real property loan clients have a 7.2% default rate, and the most applications. Car loan clients have a 9.3% default rate second highets number of applications. Education loan clients have a 9.2% default rate and third in number of applications. Wedding loans comprise the least a mount of applications and are typified by a 7.8% default rate.

<div class="alert alert-warning">
<b>Reviewer's comment v1:</b>

You have made correct conclusions based on the data that you have obtained.

On a real project, a general conclusion is probably the only thing the business will read. Therefore, it is crucial to also indicate shortly the answers to both key business questions you investigated and overall steps you accomplished in this project.
    
    
    For example:
    1. Replaced missing values in the following data with the following method.
    2. Replaced data types in the following columns.
    3. etc.
    4. etc.    

    
Additionally, I would recommend using structured project formating to keep jupyter notebook neat. For example to keep conclusions after each step as a text in a markdown cell. Here is some useful information regarding how to format text in Jupyter Notebook with Markdown.   
    
- https://www.markdownguide.org/basic-syntax/
- https://www.markdowntutorial.com/